In [1]:
import os
import czifile
import tifffile
from tqdm import tqdm
from PIL import Image

Image.MAX_IMAGE_PIXELS = None

# Path to directory containing the CZI files
czi_dir = 'czi_files'

# Directory to save the TIFF files
# creates subfolder within directory 'TIFF files'
tiff_dir = os.path.join(czi_dir, 'tiff_files')
os.makedirs(tiff_dir, exist_ok=True)

max_width = 10000  # Maximum width after downsampling
max_height = 10000  # Maximum height after downsampling
max_file_size = 1100000000 # set max file size at 1.1GB

# Get all files in directory ending with '.czi'
czi_files = [file for file in os.listdir(czi_dir) if file.endswith('.czi')]


# Loop over all files in the directory
for filename in tqdm(czi_files, desc="Converting files"):
    # Construct the full file path
    czi_file = os.path.join(czi_dir, filename)

    # check to see if file size < 1GB
    file_size = os.path.getsize(czi_file)

    # if file is small enough, proceed with conversion and downsampling
    if file_size < max_file_size:

        try:
            # Try to read the CZI file
            with czifile.CziFile(czi_file) as czi:
                img_data = czi.asarray()

        except Exception as e:
            print(f"File {filename} raised an error during processing: {str(e)}. Continuing with next file.")        
            continue
            
        # Construct the output file path (change the extension to .tiff)
        tiff_file = os.path.join(tiff_dir, os.path.splitext(filename)[0] + '.tiff')
        
        # Save as a TIFF file
        tifffile.imsave(tiff_file, img_data)

        try:
        #downsample and save over original tiff file
            with Image.open(tiff_file) as img:
                # Resize the image while preserving aspect ratio
                img.thumbnail((max_width, max_height))
                # Save the resized image
                img.save(tiff_file)
            
            print(f"Successfully downsampled {filename}.")
        
        except Exception as e:
            print(f"An error occurred while processing {filename}: {e}. Continuing with next file.")
    
    # if file is too large, skip.
    else:
        print(f"{filename} exceeds max file size. Skipping.")

print("All CZI files have been converted to TIFF and downsampled.")


Converting files:   0%|          | 0/14 [00:00<?, ?it/s]/var/folders/ky/zl0049892b138kmk5t49jgjr0000gp/T/ipykernel_12193/1849661167.py:49: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  tifffile.imsave(tiff_file, img_data)
Converting files:   7%|▋         | 1/14 [00:29<06:29, 29.95s/it]

Successfully downsampled P24-12_MM_La-1a-2024_10_28_01.czi.


Converting files:  14%|█▍        | 2/14 [01:00<06:03, 30.33s/it]

Successfully downsampled P24-12_MM_Rb-1a-2024_10_28_06.czi.


Converting files:  21%|██▏       | 3/14 [01:42<06:33, 35.82s/it]

...

Successfully downsampled P24-12_MM_Lc-1a-2024_10_28_03.czi.


Converting files:  93%|█████████▎| 13/14 [05:57<00:26, 26.89s/it]

Successfully downsampled P24-12_MM_Ra-1a-2024_10_28_05.czi.


Converting files: 100%|██████████| 14/14 [06:27<00:00, 27.64s/it]

Successfully downsampled P24-12_MM_Lb-1a-2024_10_28_02.czi.
All CZI files have been converted to TIFF and downsampled.
